
For your ingredient detection project focusing on fruits and vegetables, using the "Fruit and Vegetable Image Recognition Dataset" along with a ResNet50 model from ImageNet for transfer learning is a solid approach. Below is an example code snippet in Python using TensorFlow and Keras to set up your model. This code assumes you have already downloaded your dataset and organized it into a directory structure suitable for TensorFlow's ImageDataGenerator.




VERIFICAR LA VERSION COMPATIBLE TENSORFLOW CUDA CUDNN:
https://www.tensorflow.org/install/source#gpu




# Made with <3
# by Ivan Zepeda
# github@ijzepeda-LC

In [1]:
# Setup Constants
LEARNING_RATE=0.001
EPOCHS=50


In [2]:

from datetime import datetime
# Get date
dater= datetime.now()
# format date in DDMMYY_HHMM
DATE=dater.strftime("%d-%m_%H-%M")
print(DATE)

# Get today date and format in month/day/year




24-03_23-36


In [3]:
### Step 1: Import Necessary Libraries


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [4]:
### Step 2: Load and Preprocess the Dataset

# Adjust the data_dir to the path where your dataset is stored. The ImageDataGenerator will apply some basic data augmentation to your dataset to improve training efficacy.
 
data_dir = './FVIRD/train'  # Update this to the path of your dataset
data_dir = './FOOD/train'  # Update this to the path of your dataset

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   validation_split=0.2)  # Using 20% of data for validation

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 34672 images belonging to 88 classes.
Found 8620 images belonging to 88 classes.


In [5]:
### Step 3: Load the Pre-trained ResNet50 Model


# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
### Step 4: Add Custom Layers on Top of the Base Model

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
### Step 5: Freeze the Base Model Layers

for layer in base_model.layers:
    layer.trainable = False

In [8]:
### Step 6: Compile the Model

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
### STEP pre-7: Verify if cuda
import tensorflow as tf
tf.debugging.set_log_device_placement(True)
if tf.test.gpu_device_name():
    print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
print("Tensroflow:",tf.__version__)

"""
Version	Python version	Compiler	Build tools	cuDNN	CUDA 
tensorflow-2.12.0	3.8-3.11	GCC 9.3.1	Bazel 5.3.0	8.6	11.8
"""

Please install GPU version of TF
Tensroflow: 2.12.0


'\nVersion\tPython version\tCompiler\tBuild tools\tcuDNN\tCUDA \ntensorflow-2.12.0\t3.8-3.11\tGCC 9.3.1\tBazel 5.3.0\t8.6\t11.8\n'

In [10]:
### Step 7: Train the Model
DATE=dater.strftime("%d-%m_%H-%M")

# model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // train_generator.batch_size,
#     validation_data=validation_generator,
#     validation_steps=validation_generator.samples // validation_generator.batch_size,
#     epochs=EPOCHS)

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


# Path to save the model file
checkpoint_filepath = ".\\models\\chkpt_model-0324--{epoch:02d}-{val_accuracy:.3f}.hdf5"

# Create a ModelCheckpoint callback
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',  # Choose 'val_loss' or another metric
    mode='max',  # For validation accuracy, higher is better so we use 'max'
    save_best_only=True)

# Create an EarlyStopping callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    mode='min',  # For validation loss, lower is better so we use 'min'
    patience=5,  # Number of epochs with no improvement after which training will be stopped
    verbose=1)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, early_stopping_callback])  # Add the callbacks here


Epoch 1/50
1083/1083 [==============================] - 3204s 3s/step - loss: 2.0695 - accuracy: 0.4756 - val_loss: 1.2442 - val_accuracy: 0.6574
Epoch 2/50
1083/1083 [==============================] - 3214s 3s/step - loss: 1.4766 - accuracy: 0.5966 - val_loss: 1.1302 - val_accuracy: 0.6909
Epoch 3/50
1083/1083 [==============================] - 3373s 3s/step - loss: 1.3553 - accuracy: 0.6280 - val_loss: 1.0445 - val_accuracy: 0.7084
Epoch 4/50
1083/1083 [==============================] - 3283s 3s/step - loss: 1.2725 - accuracy: 0.6497 - val_loss: 1.0134 - val_accuracy: 0.7210
Epoch 5/50
1083/1083 [==============================] - 3177s 3s/step - loss: 1.1928 - accuracy: 0.6681 - val_loss: 0.9828 - val_accuracy: 0.7269
Epoch 6/50
1083/1083 [==============================] - 3145s 3s/step - loss: 1.1505 - accuracy: 0.6772 - val_loss: 1.0070 - val_accuracy: 0.7291
Epoch 7/50
1083/1083 [==============================] - 3142s 3s/step - loss: 1.1186 - accuracy: 0.6850 - val_loss: 0.9469 -

### Notes:

- *Dataset and File Paths*: Ensure that your dataset directory structure is compatible with flow_from_directory. Typically, this means having a subdirectory for each class in both training and validation directories.
- *Model Training*: The number of epochs, learning rate, and other hyperparameters are set to generic values. You might need to adjust these based on your specific dataset and training performance.
- *Model Fine-Tuning*: After initial training with the base model layers frozen, you can choose to unfreeze some of the top layers of the base model and continue training to potentially improve accuracy. This involves setting layer.trainable = True for the layers you wish to fine-tune and recompiling the model.

This code provides a starting point, but you'll likely need to tweak it based on your dataset's specifics and the performance you observe during training.

In [11]:

# Saving the Model
CURRENT_TRAINED_MODEL=f"model_FOOD_{DATE}_LR{LEARNING_RATE}.h5"
model.save(CURRENT_TRAINED_MODEL)

In [20]:

test_data_dir='./FOOD/test'
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 11738 images belonging to 88 classes.
367/367 [==============================] - 901s 2s/step - loss: 0.8411 - accuracy: 0.8105
Test Loss: 0.8410724997520447
Test Accuracy: 0.810529887676239


In [38]:
# Retrieve class indices
class_indices = train_generator.class_indices

# Reverse the dictionary to map indices to class names
index_to_class = {v: k for k, v in class_indices.items()}

# Print the mapping of numerical labels to category names
print("Index to class mapping:", index_to_class)

 # save index_to_class to a txt file
f = open(f"{str(CURRENT_TRAINED_MODEL.split('.')[0])}_dic.txt", "w")
f.write(str(index_to_class))
f.close()    




Index to class mapping: {0: 'Artichoke', 1: 'Avocado', 2: 'Bacon', 3: 'Banana', 4: 'Beans', 5: 'Beetroot', 6: 'Bitter Gourd', 7: 'Bread', 8: 'Broccoli', 9: 'Butter', 10: 'Cabbage', 11: 'Cauliflower', 12: 'Cheese', 13: 'Chicken', 14: 'Chickpeas', 15: 'Cinnamon', 16: 'Corn', 17: 'Cucumber', 18: 'Garlic', 19: 'Ginger', 20: 'Gourd', 21: 'Ground Meat', 22: 'Ham', 23: 'Lentils', 24: 'Meat', 25: 'Milk', 26: 'Mushroom', 27: 'Onion', 28: 'Orange', 29: 'Paneer', 30: 'Papaya', 31: 'Radish', 32: 'Sweet Potato', 33: 'Tomato', 34: 'Turnip', 35: 'apple', 36: 'asparagus', 37: 'beef meat', 38: 'bell pepper', 39: 'black beans', 40: 'blueberries', 41: 'cambray', 42: 'cantaloupe', 43: 'carrots', 44: 'celery', 45: 'chayote', 46: 'cherries', 47: 'chilli pepper', 48: 'cilantro', 49: 'eggplant', 50: 'eggs', 51: 'fish', 52: 'grapefruit', 53: 'grapes', 54: 'green beans', 55: 'guava', 56: 'jalapeno', 57: 'juice bottle', 58: 'kimchi', 59: 'kiwi', 60: 'lemon', 61: 'lettuce', 62: 'lime', 63: 'mac&cheese', 64: 'mang

In [22]:
# f = open(f"Categories_dict_mdl_{str(CURRENT_TRAINED_MODEL.split('.')[0])}.txt", "w")
# f.write(str(index_to_class))
# f.close()    


In [23]:
import os
import glob
path_test='./FOOD/test/**'
testes=glob.glob(path_test, recursive=True)

print(testes[:10])

testes2 = [x for x in testes if x.endswith('jpg')]
len(testes2)

['./FOOD/test\\', './FOOD/test\\apple', './FOOD/test\\apple\\apple__008fc187.jpg', './FOOD/test\\apple\\apple__016de5e1.jpg', './FOOD/test\\apple\\apple__01eaa43b.jpg', './FOOD/test\\apple\\apple__01eaa43b.png', './FOOD/test\\apple\\apple__02bee309.jpg', './FOOD/test\\apple\\apple__03e68c28.jpg', './FOOD/test\\apple\\apple__03e68c28.png', './FOOD/test\\apple\\apple__053abd75.jpg']


10733

In [30]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess individual images
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Paths to your test images
image_paths = testes2[::100]#['path/to/image1.jpg', 'path/to/image2.jpg', ...]

# Make predictions for each image
correct=0
incorrect=0
for image_path in image_paths:
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    predicted_class = np.argmax(prediction)
    classe=str(image_path.split('\\')[-2])
    print("Image:", image_path)
    if(classe.lower()==index_to_class[predicted_class].lower()):
        print("✅Correct", end="")
        correct=correct+1
    else:
        print("❌Incorrect", end="")
        incorrect=incorrect+1
    print("Predicted class:", predicted_class, index_to_class[predicted_class])
        
#     print("Prediction probabilities:", prediction)
    print(f"Correct:{correct}. Incorrect:{incorrect}")


1/1 [==============================] - 0s 120ms/step
Image: ./FOOD/test\apple\apple__008fc187.jpg
✅CorrectPredicted class: 35 apple
Correct:1. Incorrect:0
1/1 [==============================] - 0s 124ms/step
Image: ./FOOD/test\apple\apple__47b7877f.jpg
✅CorrectPredicted class: 35 apple
Correct:2. Incorrect:0
1/1 [==============================] - 0s 116ms/step
Image: ./FOOD/test\apple\apple__a82cb851.jpg
✅CorrectPredicted class: 35 apple
Correct:3. Incorrect:0
1/1 [==============================] - 0s 133ms/step
Image: ./FOOD/test\Artichoke\Artichoke__06d7c6a2.jpg
✅CorrectPredicted class: 0 Artichoke
Correct:4. Incorrect:0
1/1 [==============================] - 0s 124ms/step
Image: ./FOOD/test\Artichoke\Artichoke__811ca008.jpg
✅CorrectPredicted class: 0 Artichoke
Correct:5. Incorrect:0
1/1 [==============================] - 0s 117ms/step
Image: ./FOOD/test\asparagus\asparagus__06f5d2c6.jpg
✅CorrectPredicted class: 36 asparagus
Correct:6. Incorrect:0
1/1 [==============================]

In [25]:
print("Predicted class:", predicted_class, index_to_class[predicted_class])


Predicted class: 87 zuccini


In [26]:
path_ownds='C:\\Users\\ijzep\Downloads\\OwnDataset\\*'
ownds=glob.glob(path_ownds)

# Make predictions for each image
for image_path in ownds:
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    predicted_class = np.argmax(prediction)
    print("Image:", image_path)
    print("Predicted class:", predicted_class, index_to_class[predicted_class])
#     print("Prediction probabilities:", prediction)
    print()


1/1 [==============================] - 0s 287ms/step
Image: C:\Users\ijzep\Downloads\OwnDataset\apples.jpeg
Predicted class: 35 apple
1/1 [==============================] - 0s 377ms/step
Image: C:\Users\ijzep\Downloads\OwnDataset\banans.jpeg
Predicted class: 3 Banana
1/1 [==============================] - 0s 229ms/step
Image: C:\Users\ijzep\Downloads\OwnDataset\banans2.jpeg
Predicted class: 3 Banana
1/1 [==============================] - 0s 295ms/step
Image: C:\Users\ijzep\Downloads\OwnDataset\brocoli.jpeg
Predicted class: 8 Broccoli
1/1 [==============================] - 0s 270ms/step
Image: C:\Users\ijzep\Downloads\OwnDataset\cantaloupe.jpeg
Predicted class: 42 cantaloupe
1/1 [==============================] - 0s 264ms/step
Image: C:\Users\ijzep\Downloads\OwnDataset\carrots.jpeg
Predicted class: 43 carrots
1/1 [==============================] - 0s 270ms/step
Image: C:\Users\ijzep\Downloads\OwnDataset\carrots2.jpeg
Predicted class: 43 carrots
1/1 [==============================] - 0s 

In [27]:
print(list(index_to_class.values()))

['Artichoke', 'Avocado', 'Bacon', 'Banana', 'Beans', 'Beetroot', 'Bitter Gourd', 'Bread', 'Broccoli', 'Butter', 'Cabbage', 'Cauliflower', 'Cheese', 'Chicken', 'Chickpeas', 'Cinnamon', 'Corn', 'Cucumber', 'Garlic', 'Ginger', 'Gourd', 'Ground Meat', 'Ham', 'Lentils', 'Meat', 'Milk', 'Mushroom', 'Onion', 'Orange', 'Paneer', 'Papaya', 'Radish', 'Sweet Potato', 'Tomato', 'Turnip', 'apple', 'asparagus', 'beef meat', 'bell pepper', 'black beans', 'blueberries', 'cambray', 'cantaloupe', 'carrots', 'celery', 'chayote', 'cherries', 'chilli pepper', 'cilantro', 'eggplant', 'eggs', 'fish', 'grapefruit', 'grapes', 'green beans', 'guava', 'jalapeno', 'juice bottle', 'kimchi', 'kiwi', 'lemon', 'lettuce', 'lime', 'mac&cheese', 'mango', 'okra', 'paprika', 'pasta', 'peach', 'pear', 'peas', 'pineapple', 'plums', 'pomegranate', 'potatoes', 'raspberries', 'rice', 'salmon', 'soy beans', 'spaggethi', 'spinach', 'strawberries', 'tangerine', 'train', 'watermelon', 'yam', 'yogurth', 'zuccini']


In [28]:
with open(f'Categories_list_mdl_{CURRENT_TRAINED_MODEL}.txt', 'w') as file:
    file.write(str(list(index_to_class.values())))

# AGREGAR METRICAS CON LO YA ENTRENADO
## AUNQUE HAYA SALIDO ERROR

In [32]:
# Asumiendo que 'model' es tu modelo entrenado y 'test_generator' es tu generador de datos de prueba.
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)  # Convierte las probabilidades a etiquetas de clase si es necesario


367/367 [==============================] - 905s 2s/step


In [33]:
# Obtain tru labels
y_true = test_generator.classes


In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Precisión
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Precisión por clase
precision = precision_score(y_true, y_pred, average=None)
print(f'Precision by class: {precision}')

# Recall
recall = recall_score(y_true, y_pred, average='macro')
print(f'Recall: {recall}')

# F1-Score
f1 = f1_score(y_true, y_pred, average='macro')
print(f'F1-Score: {f1}')

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
print(f'Confusion Matrix:\n{cm}')


Accuracy: 0.013204975293917192
Precision by class: [0.01680672 0.02298851 0.03436426 0.00775194 0.0033557  0.03833866
 0.         0.06666667 0.02162162 0.00746269 0.01570681 0.01438849
 0.00404858 0.         0.01587302 0.         0.         0.01226994
 0.         0.01       0.00666667 0.00740741 0.01612903 0.00819672
 0.00813008 0.         0.02884615 0.00787402 0.         0.
 0.01234568 0.         0.         0.         0.         0.03313253
 0.0265252  0.01117318 0.01388889 0.         0.         0.03370787
 0.         0.         0.01851852 0.01428571 0.02272727 0.
 0.01470588 0.02512563 0.01621622 0.0173913  0.03546099 0.00645161
 0.01041667 0.02339181 0.01587302 0.00564972 0.03389831 0.01851852
 0.         0.02222222 0.01388889 0.         0.         0.03191489
 0.         0.01273885 0.02083333 0.01183432 0.0075188  0.02222222
 0.         0.         0.         0.00694444 0.02912621 0.
 0.         0.         0.         0.         0.01470588 0.
 0.00724638 0.         0.01       0.       

C:\Users\ijzep\.conda\envs\recipix\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculando precisión con el manejo de división por cero
precision = precision_score(y_true, y_pred, average=None, zero_division=1)
print(f'Precision by class: {precision}')

# Similarmente, para F1-Score, si es necesario
f1 = f1_score(y_true, y_pred, average='macro', zero_division=1)
print(f'F1-Score: {f1}')



Precision by class: [0.01680672 0.02298851 0.03436426 0.00775194 0.0033557  0.03833866
 0.         0.06666667 0.02162162 0.00746269 0.01570681 0.01438849
 0.00404858 0.         0.01587302 0.         0.         0.01226994
 0.         0.01       0.00666667 0.00740741 0.01612903 0.00819672
 0.00813008 0.         0.02884615 0.00787402 0.         0.
 0.01234568 0.         0.         0.         0.         0.03313253
 0.0265252  0.01117318 0.01388889 0.         0.         0.03370787
 0.         0.         0.01851852 0.01428571 0.02272727 0.
 0.01470588 0.02512563 0.01621622 0.0173913  0.03546099 0.00645161
 0.01041667 0.02339181 0.01587302 0.00564972 0.03389831 0.01851852
 0.         0.02222222 0.01388889 0.         0.         0.03191489
 0.         0.01273885 0.02083333 0.01183432 0.0075188  0.02222222
 0.         0.         0.         0.00694444 0.02912621 0.
 0.         0.         0.         0.         0.01470588 1.
 0.00724638 0.         0.01       0.        ]
F1-Score: 0.3631115346357012

In [37]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
import numpy as np

precision_per_class = []
for i in range(len(cm)):
    tp = cm[i, i]
    fp = np.sum(cm[:, i]) - tp
    if tp + fp == 0:
        precision = 0  # O podrías elegir manejar esto de otra manera, como con un valor predeterminado.
    else:
        precision = tp / (tp + fp)
    precision_per_class.append(precision)

print(f'Precision per class: {precision_per_class}')

Precision per class: [0.01680672268907563, 0.022988505747126436, 0.03436426116838488, 0.007751937984496124, 0.003355704697986577, 0.038338658146964855, 0.0, 0.06666666666666667, 0.021621621621621623, 0.007462686567164179, 0.015706806282722512, 0.014388489208633094, 0.004048582995951417, 0.0, 0.015873015873015872, 0.0, 0.0, 0.012269938650306749, 0.0, 0.01, 0.006666666666666667, 0.007407407407407408, 0.016129032258064516, 0.00819672131147541, 0.008130081300813009, 0.0, 0.028846153846153848, 0.007874015748031496, 0.0, 0.0, 0.012345679012345678, 0.0, 0.0, 0.0, 0.0, 0.03313253012048193, 0.026525198938992044, 0.0111731843575419, 0.013888888888888888, 0.0, 0.0, 0.033707865168539325, 0.0, 0.0, 0.018518518518518517, 0.014285714285714285, 0.022727272727272728, 0.0, 0.014705882352941176, 0.02512562814070352, 0.016216216216216217, 0.017391304347826087, 0.03546099290780142, 0.0064516129032258064, 0.010416666666666666, 0.023391812865497075, 0.015873015873015872, 0.005649717514124294, 0.0338983050847